<a href="https://colab.research.google.com/github/Pavlmits/Object-detection-yolo-v5/blob/main/CI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***YoloV5 Αναγνώριση αντικειμένων***
 Στη παρούσα εργασία θα δημιουργήσουμε ένα custom dataset με εικόνες από μήλα, πορτοκάλια και μπανάνες τα οποία θα μπορούν να αναγνωριστούν σε μία φωτογραφία ή σε ένα video. Χρησιμοποιούμε το yoloV5 και τα διαφορετικά μοντέλα που έχει για την εκπαίδευση του μοντέλου.

In [1]:
!rm -rf OIDv4_ToolKit #Διαγραφή φακέλου σε περιπτώση που υπάρχει
!git clone https://github.com/EscVM/OIDv4_ToolKit # Απόκτηση κώδικα του Open Images Dataset v4 από το GitHub για τη δημιουργία custom dataset
%cd OIDv4_ToolKit 
%pip install -r requirements.txt 
%pip install python-util

Cloning into 'OIDv4_ToolKit'...
remote: Enumerating objects: 422, done.
remote: Total 422 (delta 0), reused 0 (delta 0), pack-reused 422
Receiving objects: 100% (422/422), 34.08 MiB | 44.01 MiB/s, done.
Resolving deltas: 100% (146/146), done.
/content/OIDv4_ToolKit
     |████████████████████████████████| 3.5MB 20.2MB/s 
     |████████████████████████████████| 71kB 11.6MB/s 
     |████████████████████████████████| 7.2MB 58.0MB/s 
     |████████████████████████████████| 552kB 51.2MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: botocore 1.19.57 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.
  Found existing installation: rsa 4.6
    Uninstalling rsa-4.6:
      Successfully uninstalled rsa-4.6
  Found existing installation: docutils 0.16
    Uninstalling docutils-0.16:
      Successfully uninstalled docutils-0.16


Σε αυτό το σημείο θέλουμε να δημιουργήσουμε το δικό μας custom dataset. Το OIDv4 ToolKit είναι μία βιλβλιοθήκη της Python που βοηθά στη δημιουργία custom object detection datasets από της Amazon το OpenImages dataset. Οπότε καλούμε τη main της εφαρμογής που κατεβάσαμε με παραμέτρους  τα ονόματα των κλάσεων και τον αριθμό των αρχείων που θέλουμε να χρησιμοποιήσουμε. 

In [2]:
!python3 main.py downloader --classes Apple --type_csv all -y --limit 20000 


		   ___   _____  ______            _    _    
		 .'   `.|_   _||_   _ `.         | |  | |   
		/  .-.  \ | |    | | `. \ _   __ | |__| |_  
		| |   | | | |    | |  | |[ \ [  ]|____   _| 
		\  `-'  /_| |_  _| |_.' / \ \/ /     _| |_  
		 `.___.'|_____||______.'   \__/     |_____|
	

             _____                    _                 _             
            (____ \                  | |               | |            
             _   \ \ ___  _ _ _ ____ | | ___   ____  _ | | ____  ____ 
            | |   | / _ \| | | |  _ \| |/ _ \ / _  |/ || |/ _  )/ ___)
            | |__/ / |_| | | | | | | | | |_| ( ( | ( (_| ( (/ /| |    
            |_____/ \___/ \____|_| |_|_|\___/ \_||_|\____|\____)_|    
                                                          
        
    [INFO] | Downloading Apple.
   [ERROR] | Missing the class-descriptions-boxable.csv file.
[DOWNLOAD] | Automatic download.
...145%, 0 MB, 29985 KB/s, 0 seconds passed
[DOWNLOAD] | File class-descriptions-boxable.csv d

Η εφαρμογή OIDv4_ToolKit δημιουργεί μία δομή με τα αρχεία η οποία δεν είναι συμβατή με αυτη του yolo. Οπότε παρακάτω υλοποιούμε τα κατάλληλα βήματα για να το φέρουμε στη μορφή που θέλουμε.

In [3]:
def extract_label_name(label_name): # Μέθοδος υπέυθυνη για την εξαγωγή του label ονόματος του αρχείου
    matchObj = re.search(r'(\/)(?!.*\1)(.*).txt', label_name) # regular expression για να βρεί το τελευταίο /(.*).txt και να εξάγουμε αυτό που είναι στη παρένθεση
    return matchObj.group(2)

def find_photo_by_label_name(label_name, photos): # Μέθοδος που επιστρέφει τη φωτογραφία η οποία συνδέεται στο κατάλληλο label 
  photo_name = extract_label_name(label_name)
  pattern =".*("+photo_name +")" 
  r = re.compile(pattern)
  photo = list(filter(r.match, photos))
  return photo[0]

Το OIDv4_ToolKit δεν έχει απεικονίζει με τον ίδιο τρόπο το box που βρίσκεται το αντικείμενο στη φωτογραφία όπως τη χρειάζεται το yolo. Και παρακάτω έχουμε τη μετατρόπη αυτή.

In [4]:
def convert_to_yolo(file_label, file_name, file_img): # Μέθοδος μετατροπής σημείων που βρίσκεται το box του αντικείμενου συμβατή με το yolo.
  label_original = open(file_label, 'r')
  label_write = open(file_name, 'w')
  img = cv2.imread(file_img)
  print(label_original)
  for line in label_original:
    if line:
      # name_of_class X_min Y_min X_max Y_max
      line = line.strip()
      l = line.split(' ')
      class_name = l.pop(0)
      try:
        float(l[0])
      except ValueError:
        class_name += ' ' + l.pop(0)

      print(class_name)
      for i in range(len(l)):
        l[i] = float(l[i])
      x = ((l[2] + l[0]) / 2)
      y = ((l[3] + l[1]) / 2)
      width = (l[2] - l[0])
      height = (l[3] - l[1])

      img_width = 1 / img.shape[1]
      img_height = 1 / img.shape[0]

      x *= img_width
      y *= img_height
      width *= img_width
      height *= img_height
      print("{0} {1} {2} {3} {4}".format(class_name, x, y, width, height))
      print("{0} {1} {2} {3} {4}".format(class_name, x, y, width, height), file=label_write)
  label_write.close()


In [5]:
%cd OIDv4_ToolKit
import os # operating system library
from glob import glob # to retrieve files/pathnames matching a specified pattern
from pyutil import filereplace
from re import match
import re
import cv2

os.system("mkdir Images")
photos = glob(r'OID/**/*.jpg', recursive=True)
labels = glob(r'OID/**/*.txt', recursive=True)

for label in labels:
    filereplace(label,"Apple","0")
    filereplace(label,"Orange","1")
    filereplace(label,"Banana","2")
    photo = find_photo_by_label_name(label, photos)
    new_label = "Images/"+ extract_label_name(label)+".txt"
    print(label)
    convert_to_yolo(label, new_label, photo)
    os.system(f"cp {photo} Images/")


Streaming output truncated to the last 5000 lines.
0
0 0.4996875 0.6635755 0.999375 0.671365
OID/Dataset/train/Apple/Label/576a9033c0dfb828.txt
<_io.TextIOWrapper name='OID/Dataset/train/Apple/Label/576a9033c0dfb828.txt' mode='r' encoding='UTF-8'>
0
0 0.0473855 0.279412 0.03267900000000001 0.04248400000000002
0
0 0.06372549999999999 0.25326800000000005 0.045751 0.03267999999999999
0
0 0.0988565 0.4926475 0.116013 0.125817
0
0 0.08823500000000001 0.283497 0.039216000000000015 0.027778000000000025
0
0 0.18464049999999999 0.210784 0.032678999999999986 0.019608000000000014
0
0 0.348856 0.6919935 0.318628 0.29901900000000015
0
0 0.2818625 0.4428105 0.171569 0.12745099999999998
0
0 0.22140500000000002 0.203431 0.031046000000000018 0.01797399999999999
0
0 0.3055555 0.8120915000000001 0.05228699999999997 0.05882300000000007
0
0 0.376634 0.19035950000000001 0.18463999999999997 0.142157
0
0 0.48611150000000003 0.424837 0.35784299999999997 0.238562
0
0 0.6919935 0.6135619999999999 0.3415030000000

In [6]:
# Κατεβάζουμε το yolo v5 από το GitHub
!git clone https://github.com/ultralytics/yolov5  
%cd yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 4294 (delta 14), reused 1 (delta 0), pack-reused 4263
Receiving objects: 100% (4294/4294), 7.34 MiB | 23.13 MiB/s, done.
Resolving deltas: 100% (2902/2902), done.
/content/OIDv4_ToolKit/yolov5


In [7]:
%pip install -qr requirements.txt
%pip install wandb

     |████████████████████████████████| 276kB 8.1MB/s 
     |████████████████████████████████| 1.9MB 8.0MB/s 
     |████████████████████████████████| 133kB 55.9MB/s 
     |████████████████████████████████| 163kB 43.5MB/s 
     |████████████████████████████████| 102kB 15.3MB/s 
     |████████████████████████████████| 102kB 15.4MB/s 
     |████████████████████████████████| 71kB 11.4MB/s 
  Created wheel for watchdog: filename=watchdog-0.10.4-cp36-none-any.whl size=74842 sha256=0ff16ac63444e0befc30044f7e57a043c5d8d8b697d4ab8182ae7ca81004e426
  Stored in directory: /root/.cache/pip/wheels/9e/11/04/5160b8815b0cc7cf574bdc6d053e510169ec264c8791b4ec3a
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6490 sha256=8424fb632cc9a31b62841b3750149d773a0c4c76c953694593bce12a4a95f0e2
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp36-none-any.whl size=

In [12]:
!/content/OIDv4_ToolKit/yolov5/weights/download_weights.sh

100% 14.1M/14.1M [00:00<00:00, 20.9MB/s]

100% 41.1M/41.1M [00:01<00:00, 33.5MB/s]

100% 90.2M/90.2M [00:02<00:00, 40.0MB/s]

100% 168M/168M [00:04<00:00, 42.2MB/s]



In [8]:
import glob, os
import random

!mkdir -p training
%cd training

# put your own path here
dataset_path = '/content/OIDv4_ToolKit/Images'

# Percentage of images to be used for the validation set
percentage_test = 20

!mkdir -p data
!mkdir -p data/images
!mkdir -p data/labels
!mkdir -p data/images/train
!mkdir -p data/images/valid
!mkdir -p data/labels/train
!mkdir -p data/labels/valid

# Populate the folders
p = percentage_test/100
for pathAndFilename in glob.iglob(os.path.join(dataset_path, "*.jpg")):
    title, ext = os.path.splitext(os.path.basename(pathAndFilename))
    if random.random() <=p :
        os.system(f"cp {dataset_path}/{title}.jpg data/images/valid")
        os.system(f"cp {dataset_path}/{title}.txt data/labels/valid")
    else:
        os.system(f"cp {dataset_path}/{title}.jpg data/images/train")
        os.system(f"cp {dataset_path}/{title}.txt data/labels/train")

/content/OIDv4_ToolKit/yolov5/training


Δημιουργούμε το αρχείο dataset.yaml το οποίο είναι απαραίτητο  για την εκπαίδευση του μοντελού μας. Σε αυτό το αρχείο ορίζουμε το μέρος που βρίσκονται οι φωτογραφίες και τα labels τους, όπως επίσης και τις κλάσεις που έχουμε.

In [9]:
dataset_file = open("dataset.yaml", 'w')
print("# train and val datasets (image directory or *.txt file with image paths)", file=dataset_file)
print("train: training/data/images/train/", file=dataset_file)
print("val: training/data/images/valid/", file=dataset_file)
print("# number of classes", file=dataset_file)
print("nc: 3", file=dataset_file)
print("# class names", file=dataset_file)
print("names: ['Apple', 'Orange', 'Banana']", file=dataset_file)
dataset_file.close()

Για να χρησιμοποιήσουμε ένα από τα μοντέλα του yolo v5 κάνουμε copy το αρχείο από το φάκελο με τα μοντέλα και αντικαθιστούμε τις κατάλληλες ρυθμίσεις για το dataset μας. 

In [10]:
#3bd3b4025ac35ba0a21fa8c1107547ed84ab2185
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
%cd ..
!ls

/content/OIDv4_ToolKit/yolov5
data	    hubconf.py	README.md	  training	  utils
detect.py   LICENSE	requirements.txt  train.py	  weights
Dockerfile  models	test.py		  tutorial.ipynb


In [ ]:
os.system(f"cp /content/OIDv4_ToolKit/yolov5/models/yolov5l.yaml /content/OIDv4_ToolKit/yolov5/training")
filereplace("/content/OIDv4_ToolKit/yolov5/training/yolov5l.yaml", "nc: 80", "nc: 3")

In [ ]:
!python /content/OIDv4_ToolKit/yolov5/train.py --img 640 --batch 16 --epochs 25 --data /content/OIDv4_ToolKit/yolov5/training/dataset.yaml --cfg /content/OIDv4_ToolKit/yolov5/training/yolov5l.yaml --weights yolov5l.pt

In [ ]:
os.system(f"rm -rf /content/OIDv4_ToolKit/yolov5/training/yolov5l.yaml")
os.system(f"cp /content/OIDv4_ToolKit/yolov5/models/yolov5m.yaml /content/OIDv4_ToolKit/yolov5/training")
filereplace("/content/OIDv4_ToolKit/yolov5/training/yolov5m.yaml", "nc: 80", "nc: 3")

In [ ]:
!python /content/OIDv4_ToolKit/yolov5/train.py --img 640 --batch 16 --epochs 100 --data /content/OIDv4_ToolKit/yolov5/training/dataset.yaml --cfg /content/OIDv4_ToolKit/yolov5/training/yolov5m.yaml --weights yolov5m.pt

github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 v4.0-30-gb1cf25d torch 1.7.0+cu101 CUDA:0 (Tesla T4, 15079.75MB)

Namespace(adam=False, batch_size=16, bucket='', cache_images=False, cfg='/content/OIDv4_ToolKit/yolov5/training/yolov5m.yaml', data='/content/OIDv4_ToolKit/yolov5/training/dataset.yaml', device='', epochs=100, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[640, 640], local_rank=-1, log_artifacts=False, log_imgs=16, multi_scale=False, name='exp', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/exp', single_cls=False, sync_bn=False, total_batch_size=16, weights='yolov5m.pt', workers=8, world_size=1)
Start Tensorboard with "tensorboard --logdir runs/train", view at http://localhost:6006/
2021-01-18 21:51:29.317268: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library lib

In [ ]:
os.system(f"rm -rf /content/OIDv4_ToolKit/yolov5/training/yolov5m.yaml")
os.system(f"cp /content/OIDv4_ToolKit/yolov5/models/yolov5s.yaml /content/OIDv4_ToolKit/yolov5/training")
filereplace("/content/OIDv4_ToolKit/yolov5/training/yolov5s.yaml", "nc: 80", "nc: 3")

In [ ]:
!python /content/OIDv4_ToolKit/yolov5/train.py --img 640 --batch 16 --epochs 3 --data /content/OIDv4_ToolKit/yolov5/training/dataset.yaml --cfg /content/OIDv4_ToolKit/yolov5/training/yolov5s.yaml --weights ''

In [ ]:
os.system(f"rm -rf /content/OIDv4_ToolKit/yolov5/training/yolov5s.yaml")
os.system(f"cp /content/OIDv4_ToolKit/yolov5/models/yolov5x.yaml /content/OIDv4_ToolKit/yolov5/training")
filereplace("/content/OIDv4_ToolKit/yolov5/training/yolov5x.yaml", "nc: 80", "nc: 3")

In [ ]:
!python /content/OIDv4_ToolKit/yolov5/train.py --img 160 --batch 16 --epochs 900 --data /content/OIDv4_ToolKit/yolov5/training/dataset.yaml --cfg /content/OIDv4_ToolKit/yolov5/training/yolov5x.yaml --weights ''

In [ ]:
!python -m torch.distributed.launch --nproc_per_node 2 /content/OIDv4_ToolKit/yolov5/train.py --batch-size 64 --data /content/OIDv4_ToolKit/yolov5/training/dataset.yaml --cfg /content/OIDv4_ToolKit/yolov5/training/yolov5l.yaml --weights /content/OIDv4_ToolKit/yolov5/yolov5l.pt

In [ ]:
!/content/OIDv4_ToolKit/yolov5/weights/download_weights.sh l
!python detect.py --weights /content/OIDv4_ToolKit/yolov5/runs/train/exp/weights/best.pt --source /content/the-annoying-orange.mp4 --view-img 